In [1]:
import sys
from utils import apikey, rate, payload, regions
import utils
import os, sys, signal, threading
import requests
import json
import time
import pandas as pd
import sqlite3


In [13]:
payload

{'X-Riot-Token': 'RGAPI-abc8d04b-22f3-4e9e-ae31-bce0ca4b4f36'}

In [61]:
region = 'jp1'
queues = ['RANKED_SOLO_5x5']
queue = 'RANKED_SOLO_5x5'
baseurl = f'https://{region}.api.riotgames.com'
leagues = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'EMERALD', 'DIAMOND']
divisions = ['I','II','III','IV','V']
endpoints = {}
expert_endpoints = {
    'CHALLENGER':f'/lol/league/v4/challengerleagues/by-queue/{queue}',
    'GRANDMASTER':f'/lol/league/v4/grandmasterleagues/by-queue/{queue}',
    'MASTER':f'/lol/league/v4/masterleagues/by-queue/{queue}'
}
for league in leagues:
    for division in divisions:				
        endpoints[league+" "+division] = f'/lol/league/v4/entries/{queue}/{league}/{division}'
# add '?page={page}'


In [57]:
all_data = list()
for key in endpoints.keys():
    print(f"Starting {key}")
    l = endpoints[key]
    quit = False
    page = 1
    while not quit:    
        quit = True
        url = baseurl+l
        if key not in ['CHALLENGER', 'GRANDMASTER', 'MASTER']:
            url+=f"?page={page}"
            quit = False          
        resp = requests.get(url, headers=payload)
        if resp.ok:
            data = json.loads(resp.content)
            if len(data)>0:                        
                quit = False
                print(f"Finished page {page:03d}", end='\r')
                all_data += data
                page+=1
                time.sleep(1/rate)
            else:
                quit = True            
df = pd.DataFrame(all_data)

In [59]:
df = pd.DataFrame(all_data)

In [73]:
df.dtypes

leagueId        object
queueType       object
tier            object
rank            object
summonerId      object
summonerName    object
leaguePoints     int64
wins             int64
losses           int64
veteran           bool
inactive          bool
freshBlood        bool
hotStreak         bool
dtype: object

In [3]:
con = sqlite3.connect("data/data.db") 
cur = con.cursor()
query = '''
CREATE TABLE summoners(
    summonerId string, 
    summonerName string, 
    tier string, 
    rank string, 
    region string)
'''
cur.execute(query)
con.commit()
con.close()

In [10]:
con = sqlite3.connect("data/data.db") 
cur = con.cursor()
query = '''
ALTER TABLE summoners ADD accountId string
'''
cur.execute(query)
con.commit()
con.close()

In [11]:
con = sqlite3.connect("data/data.db")
cur = con.cursor()
query = "PRAGMA table_info(summoners);"
res = cur.execute(query)


In [12]:
res

In [13]:
res.fetchall()

[(0, 'summonerId', 'string', 0, None, 0),
 (1, 'summonerName', 'string', 0, None, 0),
 (2, 'tier', 'string', 0, None, 0),
 (3, 'rank', 'string', 0, None, 0),
 (4, 'region', 'string', 0, None, 0),
 (5, 'puuid', 'string', 0, None, 0),
 (6, 'accountId', 'string', 0, None, 0)]

In [14]:
con.commit()
con.close()